Given a set of breakpoints, there are only a certain number of possible rearrangements (i.e. moves that use two of the break points). And those breakpoints can then not be used again. So I can generate all possible combinations quite easily! 

For 8 synteny blocks

In [4]:
def generate_rearrangement_scenarios(N_BPs):

    import random
    import itertools

    BPs = [str(i) for i in range(N_BPs)]

    All_combinations = []

    combo_dict = {}

    ## find all combinations of the 8 break points

    combos = itertools.combinations(BPs,2)

    ## Im gona be super lazy and just do 4 level nested for loop . . . can come back to this if needed to make it dynamic

    all_combos = 0    

    for combination in combos:

        comboID = ",".join(combination)

        #print(combination)


        ## make a new list, with the two used BPs removed. 
        BPs_2 = []
        for i in BPs:
            if i not in combination:
                BPs_2.append(i)


        combos_2 = itertools.combinations(BPs_2,2)

        for combination_2 in combos_2:

            #print(combination, combination_2)

            comboID_2 = ",".join(combination_2)

            ## make a new list, with the two used BPs removed. 
            BPs_3 = []
            for i in BPs_2:
                if i not in combination_2:
                    BPs_3.append(i)


            combos_3 = itertools.combinations(BPs_3,2)

            for combination_3 in combos_3:

                #print(combination, combination_2, combination_3)

                comboID_3 = ",".join(combination_3)


                ## make a new list, with the two used BPs removed. 
                BPs_4 = []
                for i in BPs_3:
                    if i not in combination_3:
                        BPs_4.append(i)



                combos_4 = itertools.combinations(BPs_4,2)

                for combination_4 in combos_4:

                    #print(combination, combination_2, combination_3, combination_4)

                    comboID_3 = ",".join(combination_4)

                    all_combos += 1


                    All_combinations.append([combination, combination_2, combination_3, combination_4])

    return All_combinations
        
        


In [60]:
All_scenarios = generate_rearrangement_scenarios(8)

In [9]:
All_scenarios

[[('0', '1'), ('2', '3'), ('4', '5'), ('6', '7')],
 [('0', '1'), ('2', '3'), ('4', '5'), ('6', '8')],
 [('0', '1'), ('2', '3'), ('4', '5'), ('7', '8')],
 [('0', '1'), ('2', '3'), ('4', '6'), ('5', '7')],
 [('0', '1'), ('2', '3'), ('4', '6'), ('5', '8')],
 [('0', '1'), ('2', '3'), ('4', '6'), ('7', '8')],
 [('0', '1'), ('2', '3'), ('4', '7'), ('5', '6')],
 [('0', '1'), ('2', '3'), ('4', '7'), ('5', '8')],
 [('0', '1'), ('2', '3'), ('4', '7'), ('6', '8')],
 [('0', '1'), ('2', '3'), ('4', '8'), ('5', '6')],
 [('0', '1'), ('2', '3'), ('4', '8'), ('5', '7')],
 [('0', '1'), ('2', '3'), ('4', '8'), ('6', '7')],
 [('0', '1'), ('2', '3'), ('5', '6'), ('4', '7')],
 [('0', '1'), ('2', '3'), ('5', '6'), ('4', '8')],
 [('0', '1'), ('2', '3'), ('5', '6'), ('7', '8')],
 [('0', '1'), ('2', '3'), ('5', '7'), ('4', '6')],
 [('0', '1'), ('2', '3'), ('5', '7'), ('4', '8')],
 [('0', '1'), ('2', '3'), ('5', '7'), ('6', '8')],
 [('0', '1'), ('2', '3'), ('5', '8'), ('4', '6')],
 [('0', '1'), ('2', '3'), ('5',

So now I have all of the possible combinations of events, how do I reconstruct the sequence order and orientation given these events? 

If I treat them as indexes, I can just flip sequence right? 

Using this example . . . 

('4', '7') ('1', '5') ('2', '6') ('3', '8')

So I want to flip sequence at 4 and 7, then 1 and 5 . . .etc. But how do I flip based on the old coordinates? 


So the Seq below contains the sequence of *syntenic blocks*, and the scenario contains pairs of indexes which represent the break points. 

In [72]:
def Get_rearranged_seqs(Seq, Rearrangements, real_target_seq):

    #print(Positions)
    #print(Rearrangements)

    for i in Rearrangements:

        start = i[0]
        end = i[1]

        for j in range(len(Seq)):
            if Seq[j] == start:
                start_index = j
            elif Seq[j] == end:
                end_index = j

        if start_index < end_index:
            #end_index += 1
            inversion = Seq[start_index:end_index][::-1]
            Seq = Seq[0:start_index] + inversion + Seq[end_index:]

        elif end_index < start_index:
            #start_index += 1
            inversion = Seq[end_index:start_index][::-1]
            Seq = Seq[0:end_index] + inversion + Seq[start_index:]

        #print("Start index = %s" % start_index)
        #print("Start base = %s" % start)
        #print("End index = %s" % end_index)
        #print("End base = %s" % end)

        #print(inversion)

        #print(Positions)
        
        if Seq == real_target_seq:
            print("YES")
        
    #print(Seq, Rearrangements)
        
    return Seq

So now I need to generate the sequences for all the combinations of events . . . 

In [73]:
Rearranged_seqs = []

Seq = ["0","1","2","3","4","5","6","7"]
real_target_seq = ["7","6","2","1","0","3","4","5"]

for Scenario in All_scenarios:
    Rearranged_seqs.append((Get_rearranged_seqs(Seq, Scenario,real_target_seq),Scenario))

In [36]:
Rearranged_seqs

[(['0', '1', '2', '3', '4', '5', '6', '7', '8'],
  [('0', '1'), ('2', '3'), ('4', '5'), ('6', '7')]),
 (['0', '1', '2', '3', '4', '5', '7', '6', '8'],
  [('0', '1'), ('2', '3'), ('4', '5'), ('6', '8')]),
 (['0', '1', '2', '3', '4', '5', '6', '7', '8'],
  [('0', '1'), ('2', '3'), ('4', '5'), ('7', '8')]),
 (['0', '1', '2', '3', '6', '4', '5', '7', '8'],
  [('0', '1'), ('2', '3'), ('4', '6'), ('5', '7')]),
 (['0', '1', '2', '3', '7', '6', '4', '5', '8'],
  [('0', '1'), ('2', '3'), ('4', '6'), ('5', '8')]),
 (['0', '1', '2', '3', '5', '4', '6', '7', '8'],
  [('0', '1'), ('2', '3'), ('4', '6'), ('7', '8')]),
 (['0', '1', '2', '3', '6', '5', '4', '7', '8'],
  [('0', '1'), ('2', '3'), ('4', '7'), ('5', '6')]),
 (['0', '1', '2', '3', '6', '7', '4', '5', '8'],
  [('0', '1'), ('2', '3'), ('4', '7'), ('5', '8')]),
 (['0', '1', '2', '3', '7', '4', '5', '6', '8'],
  [('0', '1'), ('2', '3'), ('4', '7'), ('6', '8')]),
 (['0', '1', '2', '3', '7', '6', '5', '4', '8'],
  [('0', '1'), ('2', '3'), ('4', 

This is the Y order of syntenic blocks, relative to the X. 

In [68]:
for seq in Rearranged_seqs:
    if target_seq == seq[0]:
        
        print(seq)

Damn! Ok, are there any orders that get close? 

In [57]:
[i[0] for i in Rearranged_seqs]

[['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '4', '6', '5', '7'],
 ['0', '1', '2', '3', '6', '4', '5', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '5', '4', '6', '7'],
 ['0', '1', '2', '3', '6', '5', '4', '7'],
 ['0', '1', '2', '3', '6', '4', '5', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '4', '6', '5', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '6', '5', '4', '7'],
 ['0', '1', '2', '3', '5', '4', '6', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '4', '6', '5', '7'],
 ['0', '1', '2', '3', '5', '6', '4', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '5', '3', '4', '6', '7'],
 ['0', '1', '2', '6', '5', '3', '4', '7'],
 ['0', '1', '2', '4', '3', '5', '6', '7'],
 ['0', '1', '2', '3', '4', '5', '6', '7'],
 ['0', '1', '2', '3', '6', '5', '4', '7'],
 ['0', '1',

In [54]:
matched_seqs = []

for seq in Rearranged_seqs:
    matches = 0
    for i in range(len(seq)):
        if seq[0][i-1] == target_seq[i]:
            matches +=1
            
    if matches == 2:
        print(matches, seq)
            

Nothing matches - does that mean there is no way of getting to the real order? 

I will try another approach. I will generate, as above, all of the possible combinations of break point orders . . . 

I will use this to generate a sequence of fragments, where each fragment has a start and end point (called connectors)

In [38]:
Connectors = [("1A"),("1B,""2A"),("2B","3A"),("3B","4A"),("4B","5A"),("5B","6A"),("6B","7A"),("7B","8A"), ("8B")]


def generate_scenarios(connectors):

    BPs = itertools.combinations(connectors,2)

    #for i in BPs:
    #    print(i)

    ## Im gona be super lazy and just do 4 level nested for loop . . . can come back to this if needed to make it dynamic

    N_scenarios = 0    

    for BP_pair_1 in BPs:


        ## make a new list, with the two used BPs removed. 
        connectors_2 = []
        for i in connectors:
            if i not in BP_pair_1:
                connectors_2.append(i)
                
        #print(BP_pair_1)


        BPs_2 = itertools.combinations(connectors_2,2)        
        
        for BP_pair_2 in BPs_2:

            ## make a new list, with the two used BPs removed. 
            connectors_3 = []
            for i in connectors_2:
                if i not in BP_pair_2:
                    connectors_3.append(i)

            #print(BP_pair_1,BP_pair_2)
            
            BPs_3 = itertools.combinations(connectors_3,2) 
            
            
            for BP_pair_3 in BPs_3:


                ## make a new list, with the two used BPs removed. 
                connectors_4 = []
                for i in connectors_3:
                    if i not in BP_pair_3:
                        connectors_4.append(i)

                #print(BP_pair_1,BP_pair_2,BP_pair_3)

                BPs_4 = itertools.combinations(connectors_4,2) 
                

                for BP_pair_4 in BPs_4:


                    ## make a new list, with the two used BPs removed. 
                    connectors_5 = []
                    for i in connectors_4:
                        if i not in BP_pair_4:
                            connectors_5.append(i)

                    #print(BP_pair_1,BP_pair_2,BP_pair_3, BP_pair_4)

                    BPs_5 = itertools.combinations(connectors_5,2) 
                        
                    
    print(connectors_5)

generate_scenarios(Connectors)


['1A']


In [ ]:

        combos_2 = itertools.combinations(BPs_2,2)

        for combination_2 in combos_2:

            #print(combination, combination_2)

            comboID_2 = ",".join(combination_2)

            ## make a new list, with the two used BPs removed. 
            BPs_3 = []
            for i in BPs_2:
                if i not in combination_2:
                    BPs_3.append(i)


            combos_3 = itertools.combinations(BPs_3,2)

            for combination_3 in combos_3:

                #print(combination, combination_2, combination_3)

                comboID_3 = ",".join(combination_3)


                ## make a new list, with the two used BPs removed. 
                BPs_4 = []
                for i in BPs_3:
                    if i not in combination_3:
                        BPs_4.append(i)



                combos_4 = itertools.combinations(BPs_4,2)

                for combination_4 in combos_4:

                    #print(combination, combination_2, combination_3, combination_4)

                    comboID_3 = ",".join(combination_4)

                    all_combos += 1


                    All_combinations.append([combination, combination_2, combination_3, combination_4])

    return All_combinations

### Trying a searching algorithm . . . . 